# Import modules

In [ ]:
import os
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import math
%matplotlib inline
from statsmodels.graphics import tsaplots 

from leap_ec.algorithm import generational_ea
from leap_ec import ops, decoder, representation
from leap_ec.binary_rep import initializers
from leap_ec.binary_rep import problems
from leap_ec.binary_rep.ops import mutate_bitflip

from leap_ec.decoder import IdentityDecoder
import leap_ec

import copy
import random
import statistics 

# Read in data

In [ ]:
# Read in the price data
price = pd.read_csv('price.csv')
price = price.drop(columns = 'Date')
price = price.values.tolist()

In [ ]:
# Read vector of fuzzy numbers for each trading day
A = pd.read_csv('A.csv')
A = A.drop(columns = 'Date')
A = A.values.tolist()

# Individual class

In [ ]:
# This is a function that initializes a genome, used in the individual class
def genome():
    genome = []
    for i in range(0,5):
        row = []
        for j in range (0,4):
            row.append(random.uniform(0, 1))
        genome.append(row)
    return genome

In [ ]:
# Individual class
class OurIndividual:
    def __init__(self,genome,decoder,problem,initial_bank):
        self.genome = genome
        self.decoder = decoder
        self.problem = problem
        self.phenome = self.decoder.decode(self.genome)
        self.bank = initial_bank
        ##new feature
        self.fitness = 0
    def clone(self):
        # TODO: It should be a deep copy
        return self
#     def decode(self):
#         return self.phenome
    def evaluate(self, A, price, start, period):
        self.problem.evaluate(self.phenome,A, price, start, period, self.bank)
        # self.bank = self.problem.result_bank
        # return self.problem.result_bank
    
    # # These are class methods
    # @classmethod
    # def create_population(self,n,initialize,decoder,problem,initial_bank):
    #     # Population is a list of individuals
    #     population = []
    #     # Initialize n individuals and put them into the population
    #     # They are with the same decoder and problem but having different genomes
    #     for i in range(0,n):
    #         ind = OurIndividual(initialize(),decoder = decoder, problem = problem, initial_bank = initial_bank)
    #         population.append(ind)
    #     # This function returns a list of individuals of this class type
    #     return population
    
    # @classmethod
    # def evaluate_population(self,population, A, price, start, period):
    #     evaluation = []
    #     for ind in population:
    #         phenome = ind.phenome
    #         fitness = ind.problem.evaluate(phenome, A, price, start, period, ind.bank)
    #         # new feature
    #         self.fitness = fitness
    #         evaluation.append(fitness)
    #     return evaluation

    # @classmethod
    # def get_fitness(self):
    #     return self.fitness

# Population class

In [ ]:
# Mutations and crossover
mu, sigma = 0, 0.1 # mean and standard deviation
def mutate(genome):
    # Select the number of mutations that will happen to the matrix
    num_mutation = random.randrange(21)
    for x in range(0,num_mutation):
        # Generate a set of index
        i =  random.randrange(4)
        j =  random.randrange(3)
        
        # Keep the gene positive
        while True:
            # Generate a random  change
            mut = np.random.normal(mu, sigma)
            # Mutate the gene at the index
            genome[i][j] += mut
            if (genome[i][j]>= 0 and genome[i][j] <= 1): break
            else:
                genome[i][j] -= mut 
                continue
    return genome

def crossover(genome):
    # Select the number of crossovers that will happen to the matrix
    num_cross = random.randrange(21)
    for x in range(0,num_cross):
        # Generate 1st set of index
        i_1 =  random.randrange(4)
        j_1 =  random.randrange(3)
        # Generate 2nd set of index
        i_2 =  random.randrange(4)
        j_2 =  random.randrange(3)
        
        temp =  genome[i_1][j_1]
        genome[i_1][j_1] = genome[i_2][j_2]
        genome[i_2][j_2] = temp
    return genome

def one_pt_crossover(genome1, genome2):
    off_spring_1 = pd.DataFrame([])
    off_spring_2 = pd.DataFrame([])
    genome1 = pd.DataFrame(genome1)
    genome2 = pd.DataFrame(genome2)

    cut = random.randint(0,5)
    vec1 = genome1.loc[:,0]
    vec2 = genome2.loc[:,0]
    off_spring_1 = pd.DataFrame(vec1[:cut].append(vec2[cut:]))
    off_spring_2 = pd.DataFrame(vec2[:cut].append(vec1[cut:]))

    for x in range(1,4):
        cut = random.randint(0,5)
        vec1 = genome1.loc[:,x]
        vec2 = genome2.loc[:,x]
        temp_1 = pd.DataFrame(vec1[:cut].append(vec2[cut:]))
        temp_2 = pd.DataFrame(vec2[:cut].append(vec1[cut:]))
        #print('temp1', temp_1)
        off_spring_1 = np.concatenate([off_spring_1, temp_1], axis = 1)
        off_spring_2 = np.concatenate([off_spring_2, temp_2], axis = 1)
    # Change back to lists
    #off_spring_1 = off_spring_1.values.tolist()
    #off_spring_2 = off_spring_2.values.tolist()
    #print(off_spring_1)
    return off_spring_1, off_spring_2


In [ ]:
# Population class
class OurPopulation:
    def __init__(self,n,initialize,decoder,problem,initial_bank):
        # Population is a list of individuals
        population = []

        # Initialize n individuals and put them into the population
        # They are with the same decoder and problem but having different genomes
        for i in range(0,n):
            ind = OurIndividual(initialize(),decoder = decoder, problem = problem, initial_bank = initial_bank)
            population.append(ind)
        
        self.population = population
        self.fitness = []
    
    # This evaluation will inherit the individual's previous bank account
    def evaluate(self, A, price, start, period):
        population = self.population
        # Initialize a evaluation score list
        evaluation = []

        # Evaluate the fitness of each individual and append to the list
        for ind in population:
            fitness = ind.problem.evaluate(ind, A, price, start, period, ind.bank)
            ind.fitness = fitness
            evaluation.append(fitness)
        
        self.fitness = evaluation
        # Return the evaluation list
        return evaluation
    
    # This evaluation will evaluate each individual with the same bank
    def evaluate_i(self, A, price, start, period, initial_bank):
        population = self.population
        # Initialize a evaluation score list
        evaluation = []

        # Evaluate the fitness of each individual and append to the list
        for ind in population:
            # Evaluate the individual with the same bank
            fitness = ind.problem.evaluate(ind, A, price, start, period, initial_bank)
            ind.fitness = fitness
            evaluation.append(fitness)
        
        self.fitness = evaluation
        # Return the evaluation list
        return evaluation
    
    # This function evolves the population with each new individual start with initial_bank
    def evolve(self, decoder, problem, price, start, period, initial_bank):
        # initialize
        pop_obj = self
        # population = self.population

        # Get the 2 best performing individuals from orig_population
        # Evaluate the current population with the same initial bank
        pop_eval = pop_obj.evaluate(A, price, start, period)

        # The best individual and 2nd best individual
        mom_id = pop_eval.index(max(pop_eval))
        temp = pop_eval
        temp[mom_id] = 0
        dad_id = temp.index(max(temp))
        mom, dad = pop_obj.population[mom_id], pop_obj.population[dad_id]

        # Population is a list of individuals
        new_generation = []
        n = len(pop_obj.population)

        # Initialize n individuals and put them into the population
        # They are with the same decoder and problem but having different genomes
        for i in range(0,math.floor(n/2)):
            gene1, gene2 = one_pt_crossover(mom.genome, dad.genome)
            gene1 = mutate(gene1)
            gene2 = mutate(gene2)
            # Initialize 2 new individuals with initial bank
            ind1 = OurIndividual(gene1,decoder = decoder, problem = problem, initial_bank=initial_bank)
            ind2 = OurIndividual(gene2,decoder = decoder, problem = problem, initial_bank=initial_bank)
            new_generation.append(ind1)
            new_generation.append(ind2)

        # Add the new generation to the original population
        pop_obj.population.extend(new_generation)

        # Evaluate the population that have the original individuals and new individuals
        pop_obj.evaluate(A, price, start, period)

        # Sort the population to get the best performing 40 people that can stay in the population
        population = pop_obj.population
        fitness = pop_obj.fitness
        df = pd.DataFrame({'pop':population,'fit':fitness})
        df = df.sort_values(by=['fit'], ascending = False)
        population = df['pop'][:n].tolist()
        fitness = df['fit'][:n].tolist()

        pop_obj.population = population
        pop_obj.fitness = fitness
        self = pop_obj


        # # This function returns a list of individuals of this class type
        # # return new_population

# Decoder class

After analysis, the decoder class in LEAP intends to generate the phenome only using the genome of the individual. Our initial thoughts was to treat U as the phenome. However, U needed to be calculated combined with every day membership value vectors (A). Thus, our choice of decoder is the `IdentityDecoder()`, which maps the genome into itself.

# Problem class

In [ ]:
from statistics import mean, variance

# Problem class
class OurProblem():
    # This function will return the vector B
    def get_B(self, A, phenome):
        B = []
        for j in range(0,4):
            b = max(min(A[0],phenome[0][j]),
                      min(A[1],phenome[1][j]),
                      min(A[2],phenome[2][j]),
                      min(A[3],phenome[3][j]),
                      min(A[4],phenome[4][j]))
            B.append(b)
        return B
    
     # This function will return the U value
    def get_U(self, B, Lambda):
        top = 0
        bottom = 0
        for i in range(0,4):
            top += B[i]*Lambda[i]
            bottom += Lambda[i]
        U = top/bottom
        return U
    
    # This function will return the signal based on U value
    def get_signal(self, U,upper,lower):        
        if (U > upper): 
            # If U is higher than the upper threshold then buy
            signal = "B"
        elif (U < lower):
            # TODO: not very sure how it works for the sell.....
            # If U is lower than the lower threshold then sell 
            signal = "S"
        else:
            # If else, then no signal was detected
            signal = "N"
        return signal
    
    # This function will return the amount of this deal
    def  get_amount(self, signal, U,upper,lower):
        if (signal == "B"):
            # If buying then the amount is 
            amount = (U-upper)/(1-upper)*20
        elif (signal == "S"):
            # If selling then the amount is 
            amount = (U-lower)/lower*20
        else:
            # If no action then no amount
            amount = 0
        
        #if (amount <= 20):
        #    pass
        #else:
        #     amount = 20
                
        return amount
    
    # This function will find the price needed for this deal
    def get_price(self, signal,price):
        if (signal == "B"):
            # If buying then at the opening price
            price = price[0]
        elif (signal == "S"):
            # If selling then sell at the closing price
            price = price[4]
        else:
            # If no action then no price
            price = 0
        return price
    
    # This function will return the result of this deal
    def get_result(self, signal, price, amount):
        if (signal == "B"):
            result = price * amount 
        elif (signal == "S"):
            result = price * amount
        else:
            result = 0
            
        return result
            
    # This function will evaluate the fitness of a individual
    def evaluate(self, ind, A, price, start, period, initial_bank):
        # Set boundaries for buy and sell
        upper = 0.6
        lower = 0.55
        # Singleton Fuzzifier
        Lambda = [0.25, 0.5, 0.75, 1.0]
        # Get the list of vectors of fuzzy numbers (A)
        A_list = A
        # Get the list of prices with matching index of A (price)
        price_list = price
        
        # Trading start date
        start = start
        # Trading period
        period = period
        # Trading initial bank account value
        result = initial_bank
        
        # Records performance
        upside_perf = []
        downside_perf = []
        debt = initial_bank
        position = 0

        # Trade: from start date to the end of the period
        for i in range(start + 1, start+period+1):
            A_previous = A_list[i-1]
            # Calculate the U of the previous trading day
            B = self.get_B(A_previous, ind.phenome)
            U = self.get_U(B, Lambda)
            

            # The signal of this trading day
            signal = self.get_signal(U,upper,lower)
            # The amount of this tradng day
            amount = self.get_amount(signal,U,upper,lower)
            # Adjusts position held
            position += amount
            # The price of this deal for this traidng day
            price = self.get_price(signal,price_list[i])
            # Adjusts debt from the transaction
            debt -= self.get_result(signal, price, amount)
            # the Result of the new PnL
            new_result = position*price + debt
            # We keep track of the change in PnL
            daily_result = new_result - result 
            if daily_result >= 0:
                upside_perf.append(abs(daily_result/result))
            else:
                downside_perf.append(abs(daily_result/result))
            # the new Pnl replaces the old PnL
            result = new_result
        
        ind.bank = result
        
        downside_perf = np.array(downside_perf)
        upside_perf = np.array(upside_perf)

        # Sterling ratio
        # We subtract 0.02 as that is the long term interest rate from a T-Bill [supposed risk-free rate]
        rf = 0.02 *150 / 252
        ret = result
        if len(downside_perf)==0:
            sr_result = ret/rf
            upr_result = mean(upside_perf-rf)/rf
        else:
            sr_result = ret/(max(downside_perf) + rf)
            if len(upside_perf)==0:
                upr_result = -rf/math.sqrt(mean((downside_perf-rf)**2))
            else:
                upr_result = mean(upside_perf-rf)/math.sqrt(mean((downside_perf-rf)**2))
        
    
        return sr_result
    
    
    # This function prints the statistics of a individual
    def stats(self, ind, A, price, start, period, initial_bank):
        # Set boundaries for buy and sell
        upper = 0.6
        lower = 0.55
        # Singleton Fuzzifier
        Lambda = [0.25, 0.5, 0.75, 1.0]
        # Get the list of vectors of fuzzy numbers (A)
        A_list = A
        # Get the list of prices with matching index of A (price)
        price_list = price
        
        # Trading start date
        start = start
        # Trading period
        period = period
        # Trading initial bank account value
        result = initial_bank
        
        # Records performance
        upside_perf = []
        downside_perf = []
        debt = initial_bank
        position = 0

        # Trade: from start date to the end of the period
        for i in range(start + 1, start+period+1):
            A_previous = A_list[i-1]
            # Calculate the U of the previous trading day
            B = self.get_B(A_previous, ind.phenome)
            U = self.get_U(B, Lambda)
            

            # The signal of this trading day
            signal = self.get_signal(U,upper,lower)
            # The amount of this tradng day
            amount = self.get_amount(signal,U,upper,lower)
            # Adjusts position held
            position += amount
            # The price of this deal for this traidng day
            price = self.get_price(signal,price_list[i])
            # Adjusts debt from the transaction
            debt -= self.get_result(signal, price, amount)
            # the Result of the new PnL
            new_result = position*price + debt
            # We keep track of the change in PnL
            daily_result = new_result - result 
            if daily_result >= 0:
                upside_perf.append(abs(daily_result/result))
            else:
                downside_perf.append(abs(daily_result/result))
            # the new Pnl replaces the old PnL
            result = new_result
        
        ind.bank = result
        
        downside_perf = np.array(downside_perf)
        upside_perf = np.array(upside_perf)

        print("Hit Ratio: ", len(upside_perf),":",len(downside_perf))
        if (len(upside_perf) == 0):
            print("Expected Gain: 0")
        else:
            print("Expected Gain: "+"{:.2f}".format(mean(upside_perf)))
        if (len(downside_perf) == 0):
            print("Expected Loss: 0")
        else:
            print("Expected Loss: -"+"{:.2f}".format(mean(downside_perf)))

        # Sterling ratio
        # We subtract 0.02 annual rate as that is the long term interest rate from a T-Bill [supposed risk-free rate]
        rf = 0.02 * 150 / 252
        ret = result
        if len(downside_perf)==0:
            sr_result = ret/rf
            upr_result = mean(upside_perf-rf)/rf
        else:
            sr_result = ret/(max(downside_perf) + rf)
            if len(upside_perf)==0:
                upr_result = -rf/math.sqrt(mean((downside_perf-rf)**2))
            else:
                upr_result = mean(upside_perf-rf)/math.sqrt(mean((downside_perf-rf)**2))
        
    
        
        print("Sterling Ratio: "+"{:.2f}".format(sr_result))
        print("Upside Potential Ratio: "+"{:.2f}".format(upr_result))
        # Upside Potential Ratio 
        

    # This function will return if the given two are of the same fitness
    def equivalent(self, first_fitness, second_fitness):
        return first_fitness == second_fitness
    
    # This function will return which one of the fitness is better (maximum)
    def better_than(self, first_fitness, second_fitness):
        return max(first_fitness, second_fitness)
        

# Examples

In [ ]:
# Individual bank test
start = 199 # start day index
period = 150 # trading period
initial_bank = 100.0 # Initial bank account value
total_days = len(A)-period

g = genome()

ind = OurIndividual(g,decoder = IdentityDecoder(), problem = OurProblem(), initial_bank = initial_bank)
# for i in range(0,total_days):
#     start = i
#     ind.bank = initial_bank
#     ind.evaluate(A, price, start, period)
#     print(start, " ", ind.bank)
#ind.evaluate(A, price, start, period)

In [ ]:
# Population setup
start = 0 # start day index
period = 150 # trading period
initial_bank = 100.0 # Initial bank account value
n = 40 # Number of people in one population
total_days = len(A)-150 # Obtain the length of the price and A

# Initialize a population
population = OurPopulation(n,genome,IdentityDecoder(),OurProblem(),initial_bank)

# Check the first two individual's genome in this population
print("The genome of the first 2 individual's genome")
print(population.population[0].genome)
print(population.population[1].genome)

The genome of the first 2 individual's genome
[[0.45396126675938897, 0.1996463057572322, 0.2304263027143887, 0.09947679580893742], [0.9149157208585977, 0.4677876392864668, 0.45559174315348394, 0.5681332761652742], [0.9518380334148667, 0.6925380565327707, 0.8070664096849551, 0.3304282190871094], [0.7766641080855851, 0.15802727739795353, 0.6173303112658014, 0.3888583200561966], [0.8362182635544378, 0.33752099955902637, 0.04331208986826662, 0.06106630193501317]]
[[0.7059495578366947, 0.7442011547894573, 0.23133787297606012, 0.3567645442429931], [0.8548570220151063, 0.315856292955525, 0.6659425648885662, 0.10851449706713645], [0.8649192066277771, 0.5394546553798676, 0.3026627389364298, 0.9962002442903467], [0.17516282783348058, 0.13805190411229606, 0.2629678226360924, 0.5056912673893381], [0.6771058392503155, 0.45176565470364727, 0.9297033819810047, 0.7257166140871434]]


In [ ]:
# Evaluate a population with inherited bank account
population.evaluate(A, price, start, period)
evaluation = population.fitness

# Check the population's evaluation results
print("The evaluation result of the population")
print(evaluation)

The evaluation result of the population
[-74.67607757720201, -10.106065128796631, -171.16836759283632, -488.76271331786575, -105.0195611659756, -459.104591996402, -173.1049872023905, -38.73915793562669, -5577.406523471906, -1.803426508285153, -5815.345420296552, -207.82972966275918, -36.91361288806072, -15.692533890247661, -581.490778021243, -292.8935231211333, -9.011842916307556, -180.44035273464883, 17.985510147200838, -624.4972378908212, -1063.7846133491546, -239.45753461039195, -3826.7863008758422, -90.43635535023024, -77.26696909009411, -81.8752273321218, -1426.6542176899575, -271.59120686232063, -3793.462954647888, -6069.192376355081, -442.4161666921503, -324.9742707019158, -138.4541338429356, -36.07481440718074, -1073.3204607529565, -484.0285967034129, -228.33074877903584, -195.07232534066105, -1455.3612825059902, -560.6428080767198]


In [ ]:
# Evaluate a population with initial_bank
population.evaluate_i(A, price, start, period, initial_bank)
evaluation = population.fitness

# Check the population's evaluation results
print("The evaluation result of the population")
print(evaluation)

The evaluation result of the population
[-74.67607757720201, -10.106065128796631, -171.16836759283632, -488.76271331786575, -105.0195611659756, -459.104591996402, -173.1049872023905, -38.73915793562669, -5577.406523471906, -1.803426508285153, -5815.345420296552, -207.82972966275918, -36.91361288806072, -15.692533890247661, -581.490778021243, -292.8935231211333, -9.011842916307556, -180.44035273464883, 17.985510147200838, -624.4972378908212, -1063.7846133491546, -239.45753461039195, -3826.7863008758422, -90.43635535023024, -77.26696909009411, -81.8752273321218, -1426.6542176899575, -271.59120686232063, -3793.462954647888, -6069.192376355081, -442.4161666921503, -324.9742707019158, -138.4541338429356, -36.07481440718074, -1073.3204607529565, -484.0285967034129, -228.33074877903584, -195.07232534066105, -1455.3612825059902, -560.6428080767198]


In [ ]:
# Make multiple generations
for i in range(0,2):
    start = random.randint(0,total_days)
    population.evolve(IdentityDecoder(),OurProblem(),price,start,period,initial_bank)
    #print(start,population.fitness)
    # print(len(population.fitness))
    # print(len(population.population))

# Training the population

In [ ]:
# Population setup
start = 0 # start day index
period = 150 # trading period
initial_bank = 100.0 # Initial bank account value
n = 40 # Number of people in one population
test_set = 1000
testing_period = 200
total_days = len(A)-period-test_set # Obtain the length of the price and A
generations = 100 # How many generations will the population evolve

# Initialize a population
population = OurPopulation(n,genome,IdentityDecoder(),OurProblem(),initial_bank)

In [ ]:
# Make multiple generations
for i in range(0,generations):
    start = random.randint(0,total_days)
    population.evolve(IdentityDecoder(),OurProblem(),price,start,period,initial_bank)
    # print(start,population.fitness)
    # print(len(population.fitness))
    # print(len(population.population))

In [ ]:
print("The population's fitness: ",population.fitness)
print("The best trader's genome: ",population.population[0].genome)

The population's fitness:  [8404214.29100354, 8386110.713459578, 8323176.945673212, 7534310.818500593, 7500443.20594104, 7498954.759104357, 7479915.063958309, 7479915.063958309, 7479915.063958309, 7473257.599942653, 7467121.243530742, 7464399.006714451, 7426796.858413527, 7418327.944412701, 7015727.445408677, 6468070.517627708, 6412702.923287442, 6389808.785324822, 6381635.7259363895, 6357180.145167747, 6354819.78613447, 6310611.010809847, 6298949.075012775, 6274387.86750184, 6198532.162622934, 6126263.616148941, 5554441.481634657, 5533141.875489023, 5523867.897933277, 5522602.272519291, 5478870.1920455275, 5474196.645350528, 5473506.94140333, 5469898.214721521, 5469625.671417141, 5440466.287978224, 5419615.966242757, 5419467.696116774, 5400508.978832254, 5347717.037534365]
The best trader's genome:  [[0.33387894 0.44860569 0.876458   0.60684329]
 [0.43590442 0.55824656 0.47103827 0.98858742]
 [0.86558525 0.98748704 0.81284409 0.54326506]
 [0.07244312 0.59493171 0.63315992 0.55458509]


# Evaluation

In [ ]:
for i in range(5):
    # Evaluation 
    start = random.randint(len(A)-test_set-1,len(A) - testing_period - 1)
    opt_genome = population.population[0].genome
    ind = OurIndividual(opt_genome,decoder = IdentityDecoder(), problem = OurProblem(), initial_bank=100)
    print("Test attempt: ", i+1)
    evaluation = ind.problem.stats(ind, A, price, start, testing_period, initial_bank)

Test attempt:  1
Hit Ratio:  112 : 88
Expected Gain: 2.38
Expected Loss: -0.67
Sterling Ratio: 1602.62
Upside Potential Ratio: 1.31
Test attempt:  2
Hit Ratio:  102 : 98
Expected Gain: 0.49
Expected Loss: -320.99
Sterling Ratio: -0.17
Upside Potential Ratio: 0.00
Test attempt:  3
Hit Ratio:  108 : 92
Expected Gain: 0.59
Expected Loss: -1.12
Sterling Ratio: -227.00
Upside Potential Ratio: 0.12
Test attempt:  4
Hit Ratio:  105 : 95
Expected Gain: 1.56
Expected Loss: -2.66
Sterling Ratio: -84.48
Upside Potential Ratio: 0.10
Test attempt:  5
Hit Ratio:  102 : 98
Expected Gain: 0.79
Expected Loss: -1.27
Sterling Ratio: -379.27
Upside Potential Ratio: 0.14


In [ ]:
print(price)

[[-14.256422385514362, -2.6588448758376004, -14.256422385514362, -12.28734547077652, -12.28734547077652], [-11.601203192578463, -1.3364226030834885, -12.83537919624942, -13.566493109610956, -13.566493109610956], [-8.100876510386955, -4.987172122082686, -9.887371569873668, -9.778447464639484, -9.778447464639484], [-6.280688428389819, -5.9412288408802425, -8.20529664833674, -7.318828777260752, -7.318828777260752], [-6.862270897004648, -0.489805197942931, -7.4154789636167004, -6.412845802843177, -6.412845802843177], [-6.671383075634887, 5.339543892442244, -8.32479504071096, -7.198729402747296, -7.198729402747296], [-3.757814556298953, 9.486806598263833, -4.846434288724913, -5.610498114849598, -5.610498114849598], [-8.335333743902531, 9.55482641567941, -8.335333743902531, -7.066601574223893, -7.066601574223893], [-7.06046641965158, 10.23109281526938, -8.591015795651328, -9.315173112386884, -9.315173112386884], [-7.38587462198013, 8.913183874751038, -7.6091909326808915, -6.890835224993853, 

# Testing

In [ ]:
ind = OurIndividual([[0,0.6,0.5,0],
                     [0,0.33,0.33,0.33],
                     [0,0.1,1.0,0.9],
                     [0,0.44,0.5,0.1],
                     [0,0.1,0.2,0.7]],decoder = IdentityDecoder(), problem = OurProblem())

TypeError: __init__() missing 1 required positional argument: 'initial_bank'

In [ ]:
ind_copy = ind.clone()
ind_copy.genome

[[0, 0.6, 0.5, 0],
 [0, 0.33, 0.33, 0.33],
 [0, 0.1, 1.0, 0.9],
 [0, 0.44, 0.5, 0.1],
 [0, 0.1, 0.2, 0.7]]

In [ ]:
ind.phenome

[[0, 0.6, 0.5, 0],
 [0, 0.33, 0.33, 0.33],
 [0, 0.1, 1.0, 0.9],
 [0, 0.44, 0.5, 0.1],
 [0, 0.1, 0.2, 0.7]]

In [ ]:
start = 0
period = 150
initial_bank = 100.0

ind = OurIndividual(genome(),decoder = IdentityDecoder(), problem = OurProblem())

prb = OurProblem()
phenome = ind.phenome
#B = prb.get_B(A[0],phenome)
#B
result = prb.evaluate(phenome, A, price, start, period, initial_bank)
# #ind.evaluate(A, price, start, period, initial_bank)
result

57.67588549683599

In [ ]:
evaluation = ind.evaluate(A, price, start, period, initial_bank)
evaluation

57.67588549683599

In [ ]:
# Population evaluation test
n = 40

population = OurIndividual.create_population(n,genome,IdentityDecoder(),OurProblem())
#person = population[0]
pop_eval = OurIndividual.evaluate_population(population, A, price, start, period, initial_bank)
print(population[0].genome,pop_eval[0])

[[0.5838051997614904, 0.5641701895878842, 0.5128389695028838, 0.6397621141106337], [0.9638862418461862, 0.8596242105555907, 0.4939514396802914, 0.6955909212172895], [0.44773097939880224, 0.9644630865533629, 0.8568435570882578, 0.27957565405573903], [0.7233942160458424, 0.5958716263330441, 0.17741332823687528, 0.30768268160988943], [0.8384733667555498, 0.2659020014798996, 0.46386152964486604, 0.5527486630606854]] 115.70349332300668


KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
# one part cross over test
g1 = genome()
g2 = genome()
g3,g4 = one_pt_crossover(g1,g2)
print(g1)
print(g2)
print(g3)
print(g4)
type(g4)

[[0.23103235876728978, 0.5445982617238986, 0.8581571774137974, 0.2886896694358234], [0.7413326821348638, 0.27558180766721163, 0.7885257713501509, 0.7356528954184759], [0.3153184934962304, 0.28926462373279616, 0.35777327122490243, 0.08147742290199456], [0.6858166293898373, 0.167990899187353, 0.13200044412707235, 0.019122244985779857], [0.8854717070961214, 0.0007295300480353317, 0.0861478594852717, 0.4627333314705582]]
[[0.20562791657828472, 0.0616264128989884, 0.07801337131957986, 0.15181448613973225], [0.27343524237226957, 0.3639272062114314, 0.701036550706836, 0.10272435825098614], [0.9039491005087924, 0.264466641357872, 0.41503165345833526, 0.10491642989880046], [0.368498434857241, 0.3641283186728357, 0.23906919377962976, 0.5871058453865341], [0.6652575459927001, 0.48706746207732987, 0.13464718188616365, 0.5513737734885176]]
[[0.23103236 0.54459826 0.85815718 0.28868967]
 [0.74133268 0.27558181 0.78852577 0.7356529 ]
 [0.31531849 0.26446664 0.41503165 0.08147742]
 [0.68581663 0.36412

numpy.ndarray

In [ ]:
# evol test
g1 = genome()
g2 = genome()
# Population setup
start = 0 # start day index
period = 150 # trading period
initial_bank = 100.0 # Initial bank account value
n = 40 # Number of people in one population

# Initialize a population
population = OurIndividual.create_population(n,genome,IdentityDecoder(),OurProblem())
ind1 = OurIndividual(g1, decoder = IdentityDecoder(), problem = OurProblem())
ind2 = OurIndividual(g2, decoder = IdentityDecoder(), problem = OurProblem())


pop2 = evol(ind1,ind2, IdentityDecoder(), OurProblem(),population, price, start, period)


40


In [ ]:
pop = population.population
fit = population.fitness
print(type(pop),type(pop[1]),type(fit),type(fit[1]))

df = pd.DataFrame({'pop':pop,'fit':fit})
df.sort_values(by=['fit'])
pop = df['pop'].tolist()
fit = df['fit'].tolist()
print(type(pop),type(pop[1]),type(fit),type(fit[1]))

# zip_list = zip(fit, pop)
# fit, pop = sorted(zip_list, key = lambda x:x[1])[:n]
# fit, pop = zip(*sorted(zip(fit, pop)))
# print(fit)
# print(pop)

NameError: name 'population' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9433f6a9-5256-43e6-8a88-84eded61d4fc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>